In [ ]:
from datasets import load_dataset

news_datasets = load_dataset("indic_glue","bbca.hi")

In [ ]:
news_datasets

In [ ]:
news_datasets["train"].set_format("pandas")

In [ ]:
# get label counts for both classes
label_counts = news_datasets["train"]["label"].value_counts()
num_labels = (len(label_counts.keys()))

In [ ]:
label_counts

In [ ]:
news_datasets["train"].reset_format()

In [ ]:
from transformers import set_seed

# set_seed(30)
set_seed(42)

In [ ]:
from transformers import CharacterBertTokenizer, AutoModelForSequenceClassification

tokenizer = CharacterBertTokenizer(strip_accents=None, do_lower_case=None)

In [ ]:
from transformers import BertForSequenceClassification, BertConfig, CharacterBertModel, CharacterBertTokenizer

#### LOADING BERT FOR CLASSIFICATION ####

config = BertConfig.from_pretrained('bert-base-uncased', num_labels=num_labels)  # binary classification
model = BertForSequenceClassification(config=config)

In [ ]:
#### REPLACING BERT WITH CHARACTER_BERT ####

character_bert_model = CharacterBertModel.from_pretrained(
    "E:\Documents\Character Bert\Hate Speech\character-bert-hindi")
model.bert = character_bert_model

In [ ]:
def tokenize_function(example):
    #return tokenizer(example["text"], truncation=True)
    return tokenizer(example["text"], truncation=True, max_length=128)

In [ ]:
from transformers import DataCollatorWithPadding

tokenized_datasets = news_datasets.map(tokenize_function, batched=True, remove_columns=["text"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_datasets

In [ ]:
# "india": 0,
# "international": 1,
# "entertainment": 2,
# "sport":3,
# "news": 4,
# "science": 5,
# "business": 6,
# "pakistan": 7,
# "southasia":8,
# "institutional":  9,
# "social":10,
# "china": 11,
# "multimedia":  12,
# "learningenglish": 13

In [ ]:
def assign_label(example):
    mapping = {
        "india": 0,
        "international": 1,
        "entertainment": 2,
        "sport":3,
        "news": 4,
        "science": 5,
        "business": 6,
        "pakistan": 7,
        "southasia":8,
        "institutional":  9,
        "social":10,
        "china": 11,
        "multimedia":  12,
        "learningenglish": 13
    }
    example['label'] = mapping[example['label']]
    return example

In [ ]:
tokenized_datasets = tokenized_datasets.map(assign_label)
tokenized_datasets.set_format("torch")
tokenized_datasets.column_names

In [ ]:
samples = [tokenized_datasets["train"][i] for i in range(5)]
samples

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

In [ ]:
news_datasets["train"][:5]

In [ ]:
from torch.utils.data import DataLoader
# batch_size = 16
batch_size = 32

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=batch_size, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [ ]:
import torch
with torch.no_grad():
    outputs = model(**batch)
    print(outputs.loss, outputs.logits.shape)

In [ ]:
import numpy as np
import evaluate

metric_fun = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    metric_result = metric_fun.compute(references=labels, predictions=predictions)
    return {
        "accuracy": metric_result["accuracy"],
    }

In [ ]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments

# batch_size = 16
batch_size = 32
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size


training_args = TrainingArguments(
    report_to = None,
    output_dir="models/bbc-classifier",
    overwrite_output_dir=True,
    save_strategy="no",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    #learning_rate=3e-5,
    weight_decay=0.01,
    #weight_decay=0.02,
    #warmup_ratio=0.05,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6,
    #num_train_epochs=4,
    #push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [ ]:
# from datasets import concatenate_datasets

# entire_train = concatenate_datasets([tokenized_datasets["train"], tokenized_datasets["validation"]]) 

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    #train_dataset=entire_train,
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

In [ ]:
trainer.save_model()

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("models/bert-unigram-bengali-classifier")
model.to("cuda")

In [ ]:
trainer.evaluate(tokenized_datasets["validation"])